In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/EMNLP_folder_4

/content/drive/.shortcut-targets-by-id/1ANtRsCBtHNhrdxlObGcNtnE59DOMZyhS/EMNLP_folder_4


In [3]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install rouge-score -q

     |████████████████████████████████| 2.5MB 15.0MB/s 
     |████████████████████████████████| 901kB 40.4MB/s 
     |████████████████████████████████| 3.3MB 37.2MB/s 
     |████████████████████████████████| 1.2MB 16.0MB/s 


# Importing the Required Libraries

In [4]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
#import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

### Reading the predictions file and test dataset to take its arguments

In [5]:
path_dataset = '/content/drive/MyDrive/EMNLP_folder_4/dataset/'

In [6]:
f = open('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/(submitted)test_set_generations.txt','r')

preds = f.read().splitlines()

In [7]:
len(preds)

3923

In [8]:
df_test = pd.read_csv(path_dataset+'final_test.csv')

In [9]:
df_test.shape

(3923, 18)

In [10]:
#giving random key_point_id for now, these will be added later manually
kp_df = pd.DataFrame({'key_point_id': 3923*['kp_0_10'] ,'key_point':preds,'topic':df_test['topic'],'stance':df_test['stance'],'key_point_given':df_test['key_point'] })

kp_df.head()

,key_point_id,key_point,topic,stance,key_point_given
0,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."
1,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights
2,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide
3,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...
4,kp_0_10,Parents are not qualified as medical professio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."


In [11]:
final_kp_df = kp_df.drop_duplicates()

In [12]:
final_kp_df.head()

,key_point_id,key_point,topic,stance,key_point_given
0,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."
1,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights
2,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide
3,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...
4,kp_0_10,Parents are not qualified as medical professio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe..."


In [13]:
final_kp_df.shape

(405, 5)

In [14]:
final_kp_df['topic'].value_counts()

The USA is a good country to live in                            231
Routine child vaccinations should be mandatory                  109
Social media platforms should be regulated by the government     65
Name: topic, dtype: int64

In [15]:
df = final_kp_df.groupby("topic")

In [16]:
df.get_group('Routine child vaccinations should be mandatory')['stance'].value_counts()

-1    64
 1    45
Name: stance, dtype: int64

In [17]:
df.get_group('Social media platforms should be regulated by the government')['stance'].value_counts()

 1    55
-1    10
Name: stance, dtype: int64

In [18]:
df.get_group('The USA is a good country to live in')['stance'].value_counts()

 1    126
-1    105
Name: stance, dtype: int64

In [19]:
from rouge_score import rouge_scorer

In [20]:
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [21]:
scores = scorer.score(final_kp_df['key_point'][0],final_kp_df['key_point_given'][0])

In [22]:
scores['rouge1']

Score(precision=0.2222222222222222, recall=0.18181818181818182, fmeasure=0.19999999999999998)

#### Taking Rouge1 - Precision Metric for selecting the best key_point

In [23]:
def give_score(kp1,kp_given):
  scores = scorer.score(kp1,kp_given)
  return float(scores['rouge1'][0])

In [24]:
score_list = []

for i,j in zip(final_kp_df['key_point'],final_kp_df['key_point_given']):
  score_list.append(give_score(i,j))

In [25]:
final_kp_df['precision_score'] = score_list
final_kp_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,key_point_id,key_point,topic,stance,key_point_given,precision_score
0,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...",0.222222
1,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,0.000000
2,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,0.125000
3,kp_0_10,Parents will have more ability to pay-attentio...,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...,0.200000
4,kp_0_10,Parents are not qualified as medical professio...,Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...",0.111111


In [26]:
df_topic_1 = final_kp_df[final_kp_df['topic'] == 'Routine child vaccinations should be mandatory']
df_topic_2 = final_kp_df[final_kp_df['topic'] == 'Social media platforms should be regulated by the government']
df_topic_3 = final_kp_df[final_kp_df['topic'] == 'The USA is a good country to live in']

In [27]:
df_topic_1_pos_stance = df_topic_1[df_topic_1['stance'] == 1 ] 
df_topic_1_neg_stance = df_topic_1[df_topic_1['stance'] == -1 ] 
df_topic_2_pos_stance = df_topic_2[df_topic_2['stance'] == 1 ] 
df_topic_2_neg_stance = df_topic_2[df_topic_2['stance'] == -1 ] 
df_topic_3_pos_stance = df_topic_3[df_topic_3['stance'] == 1 ] 
df_topic_3_neg_stance = df_topic_3[df_topic_3['stance'] == -1 ] 

In [28]:
df_topic_1_pos_stance = df_topic_1_pos_stance.sort_values(by='precision_score', ascending=False)
df_topic_1_neg_stance = df_topic_1_neg_stance.sort_values(by='precision_score', ascending=False)
df_topic_2_pos_stance = df_topic_2_pos_stance.sort_values(by='precision_score', ascending=False) 
df_topic_2_neg_stance = df_topic_2_neg_stance.sort_values(by='precision_score', ascending=False)
df_topic_3_pos_stance = df_topic_3_pos_stance.sort_values(by='precision_score', ascending=False) 
df_topic_3_neg_stance = df_topic_3_neg_stance.sort_values(by='precision_score', ascending=False) 

In [29]:
df_topic_1_pos_stance.head()

,key_point_id,key_point,topic,stance,key_point_given,precision_score
705,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.75
703,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.60
815,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.50
704,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.50
814,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.50


In [30]:
df_topic_1_pos_stance.iloc[:,:]

,key_point_id,key_point,topic,stance,key_point_given,precision_score
705,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.750000
703,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.600000
815,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.500000
704,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
814,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.500000
816,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory...,0.454545
452,kp_0_10,Children should be vaccinated against diseases...,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable di...,0.428571
487,kp_0_10,Children should be vaccinated against diseases,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable di...,0.428571
1148,kp_0_10,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.400000
813,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.400000


In [31]:
keypoints_csv = pd.concat([df_topic_1_pos_stance.iloc[:10,:],df_topic_1_neg_stance.iloc[:10,:],df_topic_2_pos_stance.iloc[:20,:],df_topic_2_neg_stance.iloc[:20,:],df_topic_3_pos_stance.iloc[:10,:],df_topic_3_neg_stance.iloc[:10,:]])

In [32]:
keypoints_csv.shape

(70, 6)

In [33]:
keypoints_csv.head()

,key_point_id,key_point,topic,stance,key_point_given,precision_score
705,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.75
703,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,0.60
815,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to pr...,0.50
704,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.50
814,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,0.50


In [34]:
keypoints_csv= keypoints_csv.drop(['key_point_given', 'precision_score'], axis = 1) 
keypoints_csv.head()

,key_point_id,key_point,topic,stance
705,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1
703,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1
815,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1
704,kp_0_10,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1
814,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1


In [35]:
keypoints_csv.shape

(70, 4)

In [36]:
keypoints_csv = keypoints_csv.drop_duplicates()
keypoints_csv.shape

(38, 4)

#### Confirming whether we have atleast 5 keypoints belonging to each +1 and -1 stance

In [37]:
df = keypoints_csv.groupby("topic")

In [38]:
df.get_group('Routine child vaccinations should be mandatory')['stance'].value_counts()

-1    7
 1    5
Name: stance, dtype: int64

In [39]:
df.get_group('Social media platforms should be regulated by the government')['stance'].value_counts()

 1    7
-1    2
Name: stance, dtype: int64

In [40]:
df.get_group('The USA is a good country to live in')['stance'].value_counts()

 1    10
-1     7
Name: stance, dtype: int64

## Saving the key_points.csv file

In [ ]:
keypoints_csv.to_csv('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/track_2_key_points.csv')